# Family 3 saccharide BGCs - Genome Discovery Using BLAST

## Overview:
- Import 'family_3_saccharide_BGCs FASTA file.
- Using BLAST API to find neither the encapsulin proteins have a sequenced genome available.
- Output the information into a DataFrame for each encapsulin BLAST search.
    - 29 DataFrames total


## Using NCBI BLAST API search with the encapsulin proteins:

In [18]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

#define API key for increased access to 10 requests/second
api_key = "c9b038e154b263098b1022d633d445c76707"

#path to FASTA file
family_3_fasta_file_path = r'C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\family_3_saccharide_BGCs\annotated_family_3.fasta'

#read protein sequences from the FASTA file
protein_sequences = list(SeqIO.parse(family_3_fasta_file_path, "fasta"))

#perform BLASTp search for each sequence
for seq_record in protein_sequences:
    sequence = str(seq_record.seq)

    #BLAST request URL with the API key
    blastp_url = f"https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE=Proteins&DATABASE=nr&BLAST_PROGRAMS=blastp&QUERY={sequence}&api_key={api_key}"

    #perform BLAST request
    results_process = NCBIWWW.qblast(program="blastp", database="nr", sequence=sequence, url_base=blastp_url)

    #save the BLAST results
    BLASTp_results_file_name = f"{seq_record.id}_family_3_sacc._BGC_blastp_result.xml"
    with open(BLASTp_results_file_name, "w") as save_to:
        save_to.write(results_process.read())

    #close results_process
    results_process.close()

    print(f"BLAST search for {seq_record.id} completed. Results saved to {BLASTp_results_file_name}")


BLAST search for family_3_MGYP003636931262 completed. Results saved to family_3_MGYP003636931262_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP003110882604 completed. Results saved to family_3_MGYP003110882604_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP003131024615 completed. Results saved to family_3_MGYP003131024615_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP001216717877 completed. Results saved to family_3_MGYP001216717877_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP003111233400 completed. Results saved to family_3_MGYP003111233400_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP003134444350 completed. Results saved to family_3_MGYP003134444350_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP001581572508 completed. Results saved to family_3_MGYP001581572508_family_3_sacc._BGC_blastp_result.xml
BLAST search for family_3_MGYP003109322860 completed. Results 

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

## Formating the BLASTp outputs into DataFrames

In [ ]:
from Bio.Blast import NCBIXML
import pandas as pd

def parse_blast_xml(xml_file, top_hits=50):
    result_handle = open(xml_file)
    blast_records = NCBIXML.parse(result_handle)
    
    data = {'Query': [], 'Subject': [], 'Identity': [], 'E-value': []}

    for record in blast_records:
        query_id = record.query_id
        for alignment in record.alignments[:top_hits]:
            subject_id = alignment.title
            identity = alignment.hsps[0].identities
            e_value = alignment.hsps[0].expect

            data['Query'].append(query_id)
            data['Subject'].append(subject_id)
            data['Identity'].append(identity)
            data['E-value'].append(e_value)

    result_handle.close()
    return pd.DataFrame(data)

#define file paths
blast_file_paths = [ 
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP001216717877_family_3_sacc._BGC_blastp_result.xml",
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP001581572508_family_3_sacc._BGC_blastp_result.xml",
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP003110882604_family_3_sacc._BGC_blastp_result.xml",
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP003111233400_family_3_sacc._BGC_blastp_result.xml",
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP003131024615_family_3_sacc._BGC_blastp_result.xml",
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP003134444350_family_3_sacc._BGC_blastp_result.xml",
    r"C:\Users\Cameron\OneDrive - University College London\PhD\Year 1\ENCAPSULIN BIOINFORMATICS AND METAGENOMICS\encapsulin_bioinformatics_repo\BLASTp_outputs\xml_files\family_3_MGYP003636931262_family_3_sacc._BGC_blastp_result.xml"
]

#process each file 
for blast_file_path in blast_file_paths:
    #set to selecting the top_hits=50
    blast_df = parse_blast_xml(blast_file_path, top_hits=50)

    #save DataFrames to csv files
    output_file = blast_file_path.replace('.xml', '_results.csv')
    blast_df.to_csv(output_file, sep=',', index=False)
    